In [ ]:
import logging
import os
from datetime import date
from math import ceil, floor
from pathlib import Path
from typing import Literal

import geopandas as gpd
import numpy as np
import polars as pl
import xarray as xr
from dotenv import load_dotenv
from openhexa.toolbox.era5.cache import Cache
from openhexa.toolbox.era5.extract import Client, grib_to_zarr, prepare_requests, retrieve_requests

In [6]:
load_dotenv()
client = Client(url=os.getenv("CDS_URL"), key=os.getenv("CDS_API_KEY"), retry_after=10)

In [2]:
area = [16, -18, 14, -16]
start_date = date(2024, 12, 1)
end_date = date(2025, 2, 3)
variables = ["2m_temperature", "2m_dewpoint_temperature", "total_precipitation"]
output_dir = Path("data")

In [ ]:
data_vars = {
    "2m_temperature": "t2m",
    "2m_dewpoint_temperature": "d2m",
    "total_precipitation": "tp",
}

for variable in variables:
    requests = prepare_requests(
        client=client,
        dataset_id="reanalysis-era5-land",
        start_date=start_date,
        end_date=end_date,
        variable=variable,
        area=area,
        zarr_store=output_dir / variable / "store",
    )
    retrieve_requests(
        client=client,
        dataset_id="reanalysis-era5-land",
        requests=requests,
        dst_dir=output_dir / variable / "grib",
        cache=None,
    )
    grib_to_zarr(
        src_dir=output_dir / variable / "grib",
        zarr_store=output_dir / variable / "store",
        data_var=data_vars[variable],
    )